# KNIT7 One-way Latency Demo Part 1: Preparation
### Create a non-MF slice with PTP for precise time measurements (3 nodes, IPv4 network)

(This notebook is prepared specifically for KNIT7 "OWL" One-Way Latency Demo (part1) found [here](./knit7_owl_demo1.ipynb).)

For running MF timestamp or OWL (one-way latency measurement) tools, slice nodes must meet prerequisites: 

+ Git and Dockerhub must be reachable 
+ Docker has to be running
+ PTP (Precision Time Protocol) clock must be running/

This notebook creates a 3-node slice and sets up all the the above

**Last tested: 2023/09/15 by MH**

## Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

## Create an Experiment Slice

The following creates three nodes, on three PTP-capable sites, with basic NICs connected to FABRIC's FABnetv4 internet. 

In [ ]:
### TODO: Test this cell with the bleeding edge

# slice_name=f"KNIT7_slice"
# sites = []
# avoid_sites = ['NCSA']
# sites = fablib.get_random_sites(count=3,filter_function=lambda x:x['ptp_capable'] is True, avoid=(avoid_sites))

# print (f"PTP Capable sites selected are {sites}")

In [ ]:
slice_name = 'PTP_slice'

# Choose a site only if it has a PTP-flag
sites = []
non_ptp_sites = []
site_count = 3
for i in range(1,site_count+1):
    while True:
        cur_site = (fablib.get_random_sites(avoid=(sites+non_ptp_sites+['EDC','UCSD', 'MICH']))).pop()
       # print (cur_site)
        ad = fablib.get_site_advertisement(cur_site)
        if (ad.flags.ptp):
            sites.append(cur_site)
            print (f"{cur_site} selected as site{i}")
            break
        else:
            print (f"Ignoring non PTP Capable site {cur_site}")
            non_ptp_sites.append(cur_site)


node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'


In [ ]:
#Create Slice. add_fabnet() automatically adds an L3 interface on each node and assigns an IP address.

slice = fablib.new_slice(name=slice_name)

# Node1
node1 = slice.add_node(name=node1_name, site=sites[0], image='docker_rocky_8')
node1.add_fabnet()

# Node2
node2 = slice.add_node(name=node2_name, site=sites[1], image='docker_rocky_8')
node2.add_fabnet()

# Node3
node3 = slice.add_node(name=node3_name, site=sites[2], image='docker_rocky_8')
node3.add_fabnet()

#Submit Slice Request
slice.submit();

## (Optional) Observe the Slice's Attributes

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.show()
    slice.list_nodes()
    slice.list_networks()
    slice.list_interfaces()
except Exception as e:
    print(f"Exception: {e}")

## Check connectivity via Experimenter's network

The following checks Node1 --> Node2 and Node1 --> Node 3 as an example.

In [ ]:
try:
    node1 = slice.get_node(name=node1_name)   
    node2 = slice.get_node(name=node2_name)
    node3 = slice.get_node(name=node3_name)
    
    node1_addr = node1.get_interface(network_name=f'FABNET_IPv4_{node1.get_site()}').get_ip_addr()
    node2_addr = node2.get_interface(network_name=f'FABNET_IPv4_{node2.get_site()}').get_ip_addr()
    node3_addr = node3.get_interface(network_name=f'FABNET_IPv4_{node3.get_site()}').get_ip_addr()
    
    node1.execute('hostname')
    stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}') 
    stdout, stderr = node1.execute(f'ping -c 5 {node3_addr}')  
    
    node2.execute('hostname')
    stdout, stderr = node1.execute(f'ping -c 5 {node3_addr}')  

    
except Exception as e:
    print(f"Exception: {e}")

# Prepare each node for time precision experiments

In [ ]:
nodes = slice.get_nodes()

## Optimize repositories based on management network type (ipv4 vs ipv6)


In [ ]:
nodes = slice.get_nodes()
for node in nodes:
    IPv6Management = False
    ip_proto_index = "4"
    commands = "sudo ip -6 route del default via `ip -6 route show default|grep fe80|awk '{print $3}'` > /dev/null 2>&1"
    if node.validIPAddress(node.get_management_ip()) == "IPv6":
        IPv6Management = True
        ip_proto_index = "6"
    if [ele for ele in ["rocky", "centos"] if (ele in node.get_image())]:
        commands = (
            f'sudo echo "max_parallel_downloads=10" |sudo tee -a /etc/dnf/dnf.conf;'
            f'sudo echo "fastestmirror=True" |sudo tee -a /etc/dnf/dnf.conf;'
            f'sudo echo "ip_resolve='
            + ip_proto_index
            + '" |sudo tee -a /etc/dnf/dnf.conf;'
        )
    elif [ele for ele in ["ubuntu", "debian"] if (ele in node.get_image())]:
        commands = (
            'sudo echo "Acquire::ForceIPv'
            + ip_proto_index
            + ' "true";" | sudo tee -a /etc/apt/apt.conf.d/1000-force-ipv'
            + ip_proto_index
            + "-transport"
        )
    if commands:
        stdout, stderr = node.execute(commands, quiet=True)
        print (f"Optimizing Repos for {node.get_name()}")
        #print (f"STDOUT: {stdout}")
        if stderr:
            print (f"STDERR: {stderr}")

## Set up PTP (Precision Time Protocol)

This block can take a while to execute. If successful, it will print `Installation of PTP Completed`.

In [ ]:
pre_requisites = None

clone_instructions = f"""
cd /tmp/;git clone --filter=blob:none --no-checkout --depth 1 --sparse https://github.com/fabric-testbed/MeasurementFramework.git;
cd /tmp/MeasurementFramework;git sparse-checkout add instrumentize/ptp/ansible;git checkout;
"""

ansible_instructions = f"""
cd /tmp/MeasurementFramework/instrumentize/ptp/ansible;
ansible-playbook --connection=local --inventory 127.0.0.1, --limit 127.0.0.1 playbook_fabric_experiment_ptp.yml;
"""

#Create execute threads
execute_threads = {}

for node in nodes:
    if [ele for ele in ["rocky", "centos"] if (ele in node.get_image())]:
        pre_requisites = f"""
        sudo dnf -y install epel-release ; sudo dnf -y install ansible git; mkdir /tmp/ptp_ansible/;
        """
    elif [ele for ele in ["ubuntu", "debian"] if (ele in node.get_image())]:
        pre_requisites = f"""sudo apt-get update;sudo apt-get -y install ansible git;"""
    else:
        pre_requisites = None
    print (f"Installing PTP on {node.get_name()}")
    execute_threads[node] = node.execute_thread(\
                f"{pre_requisites}"\
                f"{clone_instructions}"\
                f"{ansible_instructions}",\
                output_file=f"/tmp/{node.get_name()}_ptpinstall.log"\
                )

    #Wait for results from threads
for node,thread in execute_threads.items():
    print(f"Waiting for result from node {node.get_name()}")
    stdout,stderr = thread.result()
    #print(f"stdout: {stdout}")
    #print(f"stderr: {stderr}")
    #node.execute(f"{pre_requisites}"\
    #             f"{ansible_instructions}"\
    #            )

print (f"Installation of PTP Completed\n\n")

## Start Docker and verify it is running

`docker ps` should print `CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES`

In [ ]:
for node in nodes:
    node.execute("sudo systemctl start docker")
    node.execute("sudo systemctl enable docker")
    node.execute("sudo usermod -aG docker rocky")
    
    print(f"\n Verify installtion... on {node.get_name()}")
    node.execute("docker ps")

# (Optional) Extend the slice (Add 14 days)

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Lease End         : {slice.get_lease_end()}")
       
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
import datetime

#Extend slice
end_date = (datetime.datetime.now().astimezone() + datetime.timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)
    slice.renew(end_date)
    print(f"New lease end date : {slice.get_lease_end()}")
    
except Exception as e:
    print(f"Exception: {e}")

# Delete the Slice

In [ ]:
# try:
#     slice = fablib.get_slice(name=slice_name)
#     slice.delete()
# except Exception as e:
#     print(f"Exception: {e}")